# Bandit Algorithms

Các bài toán Máy đánh bạc được giới thiệu bởi William R. Thompson vào năm 1933. Để mô phỏng tình huống học tập ở con người, một máy đánh bạc 2 cần gạt được xét đến, nơi người chơi chọn kéo tay trái hoặc tay phải, mỗi tay cho một phần thưởng ngẫu nhiên với phân phối phần thưởng là không biết đối với người chơi.

Có nhiều lý do để quan tâm đến các bài toán Máy đánh bạc. Quyết định trong điều kiện không chắc chắn là một thách thức mà tất cả chúng ta đều đối mặt, và Máy đánh bạc cung cấp một mô hình đơn giản của tình huống khó xử này. Đây cũng là một trường hợp đặc biệt để dẫn dắt chúng ta đến với chủ đề Học tăng cường (Reinforcement Learning). Học tăng cường liên quan đến việc một tác nhân học cách hành động trong môi trường để tối đa hóa phần thưởng dài hạn, và các bài toán Máy đánh bạc cung cấp một mô hình đơn giản và hiệu quả để nghiên cứu và phát triển các thuật toán học tăng cường.

## Vấn đề cổ điển

Hãy tưởng tượng bạn đang chơi một máy đánh bạc hai cần gạt và bạn đã kéo mỗi tay 5 lần, kết quả là các phần thưởng như sau:

<div align="center">
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    table {
      width: 60%;
      border-collapse: collapse;
      table-layout: fixed;
    }
    th, td {
      border: 1px solid black;
      text-align: center;
      padding: 10px;
    }
  </style>
</head>
<body>
  <table>
    <tr>
      <th>Vòng</th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
      <th>9</th>
      <th>10</th>
    </tr>
    <tr>
      <td>TRÁI</td>
      <td>0</td>
      <td></td>
      <td>10</td>
      <td>0</td>
      <td></td>
      <td>0</td>
      <td></td>
      <td></td>
      <td></td>
      <td>10</td>
    </tr>
    <tr>
      <td>PHẢI</td>
      <td></td>
      <td>10</td>
      <td></td>
      <td></td>
      <td>0</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td></td>
    </tr>
  </table>
</body>
</html>
</div>

Cánh tay trái có vẻ hoạt động tốt hơn một chút. Phần thưởng trung bình cho cánh tay này là 4, trong khi phần thưởng trung bình cho cánh tay phải chỉ là 2. 

Giả sử bạn còn 10 lượt chơi nữa. Chiến lược của bạn là gì? Bạn sẽ tiếp tục kéo cánh tay trái, bỏ qua cánh tay phải? Hay bạn sẽ cho rằng hiệu suất kém của cánh tay phải là do vận xui và thử nó thêm vài lần nữa? Bao nhiêu lần nữa? Điều này minh họa một trong những quan tâm chính trong các bài toán Máy đánh bạc. Chúng nắm bắt tình huống khó xử cơ bản mà người học gặp phải khi chọn giữa các tùy chọn không chắc chắn. Có nên khám phá một tùy chọn trông kém hấp dẫn hơn hoặc khai thác bằng cách chọn tùy chọn hiện tại trông có vẻ tốt nhất? Tìm ra sự cân bằng đúng đắn giữa khám phá và khai thác là trọng tâm của tất cả các bài toán Máy đánh bạc.

## Phát biểu bài toán

Một bài toán Máy đánh bạc là một trò chơi tuần tự giữa **người học** (learner) và **môi trường** (environment). Trò chơi này diễn ra qua $n$ vòng, trong đó $n$ là một số tự nhiên dương gọi là **tầm nhận thức** (horizon). Trong mỗi vòng $t\in[n]$, người học đầu tiên chọn một hành động $A_t$ từ tập hợp các hành động $\mathcal{A}$, và môi trường sau đó tiết lộ một phần thưởng $X_t\in\mathbb{R}$.

Thách thức cơ bản trong các bài toán Máy đánh bạc là môi trường không được biết trước đối với người học. Tất cả những gì người học biết là môi trường thật sự nằm trong một tập hợp $\mathcal{E}$ gọi là **lớp môi trường** (environment class). Khái niệm này để chỉ tập hợp các môi trường khả dĩ mà người học có thể đối mặt. 

> Trong các tài liệu, các hành động thường được gọi là "arms" (cần gạt). Bài toán được gọi là *$k$-armed bandits* khi số lượng hành động là $k$, và *multi-armed bandits* khi số lượng hành động ít nhất là 2. 

Dĩ nhiên, người học không thể nhìn trước tương lai khi chọn hành động, nghĩa là $A_t$ chỉ nên phụ thuộc vào **lịch sử** $H_{t-1}=(A_1,X_1,A_2,X_2,\ldots,A_{t-1},X_{t-1})$. Một **chính sách** (policy) là một ánh xạ từ lịch sử đến hành động: Người học áp dụng một chính sách để tương tác với môi trường. Một môi trường là một ánh xạ từ chuỗi lịch sử kết thúc bằng hành động đến phần thưởng. Mục tiêu phổ biến nhất của người học là chọn các hành động dẫn đến tổng phần thưởng lớn nhất có thể qua tất cả $n$ vòng, đó là $\sum_{t=1}^n X_t$.

Câu hỏi tiếp theo là làm thế nào để đánh giá một người học. Chúng ta sẽ thảo luận một số biện pháp, nhưng hầu hết nỗ lực của chúng ta hướng tới việc hiểu đúng thước đo hối tiếc (regret). Đây là thước đo mất mát của người học do không chọn được hành động tối ưu ngay từ đầu.

## Các định nghĩa phổ biến

**Định nghĩa 1.** Lượng **hối tiếc** (regret) của người học so với một chính sách $\pi$ (không nhất thiết phải là chính sách người học đang tuân theo) là sự khác biệt giữa tổng phần thưởng kỳ vọng khi sử dụng chính sách $\pi$ trong $n$ vòng và tổng phần thưởng kỳ vọng mà người học thu thập được qua $n$ vòng. Lượng hối tiếc so với một tập hợp chính sách $\Pi$ là lượng hối tiếc tối đa so với bất kỳ chính sách nào $\pi\in\Pi$ trong tập hợp.

Tập hợp $\Pi$ thường được gọi là **lớp đối thủ** (competitor class). Nói cách khác, lượng hối tiếc đo lường hiệu suất của người học so với chính sách tốt nhất trong lớp đối thủ. Chúng ta thường đo lường lượng hối tiếc so với một tập hợp chính sách $\Pi$ đủ lớn để bao gồm chính sách tối ưu cho mọi môi trường trong $\mathcal{E}$. Trong trường hợp này, lượng hối tiếc đo lường mất mát của người học so với chính sách tối ưu.

**Ví dụ 1.** Giả sử tập hợp hành động là $\mathcal{A} = \{1, 2, \ldots, k\}$. Một môi trường được gọi là *máy đánh bạc ngẫu nhiên Bernoulli* (stochastic Bernoulli bandit) nếu phần thưởng $X_t \in \{0, 1\}$ có giá trị nhị phân và tồn tại một vector $\mu \in [0, 1]^k$ sao cho xác suất $X_t = 1$ khi người học chọn hành động $A_t = a$ là $\mu_a$. 

> Xét một bài toán Máy đánh bạc Bernoulli với 3 lựa chọn (arms). Mỗi lần chọn một máy, bạn nhận được phần thưởng $0$ hoặc $1$.
> - **Môi trường cụ thể:** Là đối tác của người học, được đặc trưng bởi xác suất nhận phần thưởng $1$ cho mỗi lựa chọn, chẳng hạn là $0.2$, $0.5$, và $0.8$.  
> - **Lớp môi trường:** Tập hợp tất cả các phân phối Bernoulli có thể cho xác suất nhận phần thưởng $1$ ở mỗi lựa chọn. Trong trường hợp này, lớp môi trường có thể là tất cả các vector xác suất $\mu = (\mu_1, \mu_2, \mu_3)$, trong đó $\mu_i \in [0, 1], i\in[3]$.

Lớp các Máy đánh bạc Bernoulli là tập hợp tất cả các máy đánh bạc có phần thưởng là biến ngẫu nhiên tuân theo phân phối Bernoulli, được đặc trưng bởi các vector giá trị trung bình. Nếu bạn biết vector giá trị trung bình liên quan đến môi trường, thì chính sách tối ưu là thực hiện hành động cố định $a^* = \arg\max_{a \in \mathcal{A}} \mu_a$. Điều này có nghĩa là đối với vấn đề này, lớp đối thủ là tập hợp các chính sách cố định $\Pi = \{\pi_1, \ldots, \pi_k\}$, trong đó $\pi_i$ chọn hành động $i$ trong mọi vòng. 

Lượng hối tiếc qua $n$ vòng là:

\begin{align*}
R_n = n \max_{a \in \mathcal{A}} \mu_a - \mathbb{E} \left[ \sum_{t=1}^{n} X_t \right]
\end{align*}

trong đó kỳ vọng được tương ứng theo tính ngẫu nhiên trong môi trường và chính sách. Số hạng đầu tiên trong biểu thức này là phần thưởng kỳ vọng tối đa khi sử dụng bất kỳ chính sách nào. Số hạng thứ hai là phần thưởng kỳ vọng mà người học thu thập được.

Đối với một chính sách cố định và lớp đối thủ cố định, lượng hối tiếc phụ thuộc vào môi trường. Các môi trường có hối tiếc lớn là những nơi người học hành động tệ hơn. Dĩ nhiên, trường hợp lý tưởng là lượng hối tiếc nhỏ đối với tất cả các môi trường. Lượng hối tiếc xấu nhất (worst-case regret) là lượng hối tiếc tối đa trên tất cả các môi trường có thể xảy ra.

Một trong những câu hỏi cốt lõi trong nghiên cứu bài toán Máy đánh bạc là hiểu sự tăng trưởng của lượng hối tiếc khi $n$ tăng. Một người học giỏi đạt được lượng hối tiếc dưới tuyến tính. Gọi $R_n$ là lượng hối tiếc qua $n$ vòng, điều này có nghĩa là $R_n = o(n)$ hoặc tương đương là:

\begin{align*}
\lim_{n \to +\infty} \frac{R_n}{n} = 0.
\end{align*}

Dĩ nhiên, chúng ta có thể yêu cầu nhiều hơn nữa. Trong những trường hợp nào thì $R_n = \mathcal{O}(\sqrt{n})$ hoặc $R_n = \mathcal{O}(\log(n))$? Và các hằng số đứng đầu là gì? Làm thế nào để lượng hối tiếc phụ thuộc vào môi trường cụ thể trong đó người học tìm thấy mình? Chúng ta sẽ khám phá chi tiết hơn rằng đối với lớp môi trường trong Ví dụ 1, lượng hối tiếc trong trường hợp tồi tệ nhất là ít nhất $\Omega(\sqrt{n})$ và rằng có các chính sách thử nghiệm tốt nhất cho $R_n = \mathcal{O}(\sqrt{n})$.

> Một lớp môi trường lớn tương ứng với ít kiến thức hơn của người học. Một lớp đối thủ lớn có nghĩa là lượng hối tiếc là một tiêu chí đòi hỏi khắt khe hơn. Đôi khi cần thận trọng khi chọn các tập hợp này một cách hợp lý để đảm bảo rằng (a) các đảm bảo về lượng hối tiếc có ý nghĩa và (b) tồn tại các chính sách làm cho lượng hối tiếc nhỏ.

## Các giả định cơ bản

Một máy đánh bạc ngẫu nhiên là một tập hợp các phân phối $\nu = (P_a : a \in \mathcal{A})$, trong đó $\mathcal{A}$ là tập hợp các hành động có sẵn. Người học và môi trường tương tác tuần tự qua $n$ vòng. Trong mỗi vòng $t \in [n]$, người học chọn một hành động $A_t \in \mathcal{A}$, và hành động này được đưa vào môi trường. Môi trường sau đó chọn một phần thưởng $X_t \in \mathbb{R}$ từ phân phối $P_{A_t}$ và tiết lộ $X_t$ cho người học. 

Sự tương tác giữa người học (hoặc chính sách) và môi trường tạo ra một phân phối xác suất trên chuỗi các kết quả $A_1, X_1, A_2, X_2, \ldots, A_n, X_n$. Thường thì $n$ là hữu hạn, nhưng đôi khi chúng ta cho phép tương tác tiếp tục vô thời hạn ($n = + \infty$). Chuỗi các kết quả phải thỏa mãn các giả định sau:

(a) Phân phối có điều kiện của phần thưởng $X_t$ cho trước $A_1, X_1, \ldots, A_{t-1}, X_{t-1}, A_t$ là $P_{A_t}$.

(b) Luật có điều kiện của hành động $A_t$ cho trước $A_1, X_1, \ldots, A_{t-1}, X_{t-1}$ là $\pi_t(\cdot \mid A_1, X_1, \ldots, A_{t-1}, X_{t-1})$, trong đó $\pi_1, \pi_2, \ldots$ là một chuỗi các nhân xác suất đặc trưng cho người học. Yếu tố quan trọng nhất của giả định này là người học không thể sử dụng các quan sát tương lai trong các quyết định hiện tại.

Mục tiêu của người học là tối đa hóa tổng phần thưởng $S_n=\sum_{t=1}^n X_t$, đây là một đại lượng ngẫu nhiên phụ thuộc vào các hành động của người học và các phần thưởng được môi trường chọn. 

## Các lớp môi trường

Người học thường chỉ có thông tin một phần về $\nu$, điều mà chúng ta biểu thị bằng cách định nghĩa một tập hợp các máy đánh bạc mà $\nu\in\mathcal{E}$ được đảm bảo. Tập hợp $\mathcal{E}$ chính là lớp môi trường. Chúng ta phân biệt giữa lớp môi trường có cấu trúc (structured) và phi cấu trúc (unstructured).

**Định nghĩa 2.** Một lớp môi trường $\mathcal{E}$ là phi cấu trúc nếu $\mathcal{A}$ là hữu hạn và có các tập hợp phân phối $\mathcal{M}_a$ cho mỗi $a\in\mathcal{A}$ thì:
\begin{align*}
\mathcal{E} = \{\nu=(P_a:a\in\mathcal{A}):P_a\in\mathcal{M}_a, \forall a\in\mathcal{A}\}
\end{align*}
hay ngắn gọn hơn, $\mathcal{E}=\times_{a\in\mathcal{A}} \mathcal{M}_a$. Cấu trúc tích này có nghĩa là bằng cách chơi hành động $a$, người học không thể suy ra bất cứ điều gì về các phân phối của các hành động $b\neq a$.

Một số lựa chọn điển hình của lớp môi trường phi cấu trúc được liệt kê trong bảng dưới đây. Tất nhiên, đây không phải là những lựa chọn duy nhất, và người đọc có thể tìm thấy các cách khác để xây dựng thêm, ví dụ: bằng cách cho phép một số arms là Bernoulli và một số là Gaussian, hoặc có các phần thưởng được phân phối theo mũ, hoặc phân phối Gumbel, hoặc thuộc về họ phi tham số (non-parametric).

<div align="center">

| Tên                 | Ký hiệu                          | Định nghĩa                                                                                 |
|----------------------|---------------------------------|--------------------------------------------------------------------------------------------|
| Bernoulli            | $\mathcal{E}_B^k$               | $\{(\mathcal{B}(\mu_i))_i : \mu \in [0, 1]^k\}$                                            |
| Uniform              | $\mathcal{E}_U^k$               | $\{(\mathcal{U}(a_i, b_i))_i : a, b \in \mathbb{R}^k \text{ với } a_i \leq b_i, \forall i\}$ |
| Gaussian (known var.)| $\mathcal{E}_{\mathcal{N}}^k(\sigma^2)$ | $\{(\mathcal{N}(\mu_i, \sigma^2))_i : \mu \in \mathbb{R}^k\}$                             |
| Gaussian (unknown var.) | $\mathcal{E}_{\mathcal{N}}^k$    | $\{(\mathcal{N}(\mu_i, \sigma_i^2))_i : \mu \in \mathbb{R}^k \text{ và } \sigma^2 \in [0, \infty)^k\}$ |
| Finite variance      | $\mathcal{E}_{\mathbb{V}}^k(\sigma^2)$     | $\{(P_i)_i : \mathbb{V}_{X \sim P_i}[X] \leq \sigma^2, \forall i\}$ |
| Finite kurtosis      | $\mathcal{E}_{\text{Kurt}}^k(\kappa)$ | $\{(P_i)_i : \text{Kurt}_{X \sim P_i}[X] \leq \kappa, \forall i\}$                 |
| Bounded support      | $\mathcal{E}_{[a, b]}^k$        | $\{(P_i)_i : \text{Supp}(P_i) \subseteq [a, b]\}$                                         |
| Subgaussian          | $\mathcal{E}_{\text{SG}}^k(\sigma^2)$ | $\{(P_i)_i : P_i \text{ is } \sigma\text{-subgaussian},\forall i\}$                      |

</div>

**Định nghĩa 3.** Các lớp môi trường không phải là không có cấu trúc được gọi là có cấu trúc.

**Ví dụ 2.** Giả sử $\mathcal{A} = \{1, 2\}$ và $\mathcal{E} = {(\mathcal{B}(\theta), \mathcal{B}(1 - \theta)) : \theta \in [0, 1]}$. Trong lớp môi trường này, người học không biết giá trị trung bình của bất kỳ arm nào, nhưng có thể học được giá trị trung bình của cả hai arm bằng cách chỉ chơi một arm. Kiến thức về cấu trúc này thay đổi đáng kể độ khó của việc học trong vấn đề này.

**Ví dụ 3.** Giả sử $\mathcal{A} \subset \mathbb{R}^d$ và $\theta \in \mathbb{R}^d$ và
\begin{align*}
\nu_{\theta} = \left(\mathcal{N}(\langle a,\theta\rangle,1):a\in\mathcal{A}\right) \, \text{và} \, \mathcal{E}=\{\nu_{\theta}:\theta\in\mathbb{R}^d\}
\end{align*}
Trong lớp môi trường này, phần thưởng của một hành động tuân theo phân phối Gauss, và giá trị trung bình của nó được cho bởi tích vô hướng giữa hành động và một số tham số chưa biết. Lưu ý rằng ngay cả khi $\mathcal{A}$ rất lớn, người học có thể suy ra môi trường thật bằng cách chỉ chơi $d$ hành động mà span $\mathbb{R}^d$.

## Lượng hối tiếc

Chúng ta đã định nghĩa không chính thức lượng hối tiếc là sự thiếu hụt mà người học phải chịu so với chính sách tối ưu. Giả sử $\nu = (P_a : a \in \mathcal{A})$ là đặc trưng của một bài toán máy đánh bạc nào đó và định nghĩa:

\begin{align*}
\mu_a(\nu) = \int_{-\infty}^{\infty} x \, dP_a(x)
\end{align*}

Sau đó, hãy để $\mu^*(\nu) = \max_{a \in \mathcal{A}} \mu_a(\nu)$ là giá trị trung bình lớn nhất của tất cả các máy.

Chúng ta giả định rằng $\mu_a(\nu)$ tồn tại và là hữu hạn cho tất cả các hành động và rằng $\arg\max_{a \in \mathcal{A}} \mu_a(\nu)$ là không rỗng.

Lượng hối tiếc của chính sách $\pi$ trên bài toán $\nu$ là

\begin{align*}
R_n(\pi, \nu) = n \mu^*(\nu) - \mathbb{E} \underbrace{\left[ \sum_{t=1}^{n} X_t \right]}_{S_n}
\end{align*}

trong đó kỳ vọng được tính theo xác suất đo trên các kết quả do sự tương tác của $\pi$ và $\nu$ tạo ra. Việc giảm thiểu lượng hối tiếc tương đương với việc tối đa hóa kỳ vọng của $S_n$.

Lượng hối tiếc luôn không âm, và đối với mỗi bài toán $\nu$, tồn tại một chính sách $\pi$ mà đối với chính sách đó, lượng hối tiếc tiệm cận về $0$.
